In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.feature_selection import VarianceThreshold
import category_encoders as ce

from sklearn.model_selection import GridSearchCV

# Load and merge the data

In [2]:
train = pd.read_csv("train.csv").set_index('index')
test = pd.read_csv("test.csv").set_index('index')

train_ind = train.index
test_ind = test.index


card = pd.read_csv("card.csv")
user = pd.read_csv("user.csv")

## drop unuse train col
# train_col = train.columns
# train_col =train_col.drop('index' )
# train_col =train_col.drop('card_id' )

## -merge user
train = train.merge(user, how='left')
test = test.merge(user, how='left')

# # ## --merge card
# train = train.merge(card)
# test = test.merge(card)

train = train.set_index(train_ind)
test = test.set_index(test_ind)

# Preprocessor

In [3]:
class preprocessor:
    def __init__(self ):
#         self.train_col = train_col
#         self.test_col = self.train_col.drop('is_fraud?')
        a =1
    #def fit(self, data):
        self.cate = ['']
        self.target = 'is_fraud?'
        self.target_emb_feature = ['merchant_city']
        self.drop_user_info = ['birth_year' , 'birth_month' ]
        self.drop_card_info = [ 'expires' ,'acct_open_date' ,'card_brand' , 'card_type'  , 'has_chip' ]
        self.user_obj_to_cat = ['user_id','gender' , 'address','city' ,'state' , 'zipcode' ]
        self.card_obj_to_cat =  [ ]
        
        self.user_emb_feature = []#  ,'mcc','merchant_id' 'merchant_city' , 'merchant_state', 'merchant_state'
        self.money_col = ['amount','per_capita_income_zipcode' ,'yearly_income_person','total_debt'] #,'credit_limit'
        self.date_col = []
        self.GLMM = {}
        
    
        
    def common_transform(self, data):
        x = data.copy()
        x = x.fillna(0)
        ### ---------- chage to category
        for feat in self.user_obj_to_cat:
            x[feat] = x[feat].astype('category')
        for feat in self.card_obj_to_cat:
            x[feat] = x[feat].astype('category')
        x['zip'] = x['zip'].astype('category')
        x['mcc'] = x['mcc'].astype('category')
        #### drop features---------------
        x = x.drop(columns = 'card_id')
        
        x = x.drop(columns = self.drop_user_info)
#         x = x.drop(columns = self.drop_card_info)
        ## one hot errors
        x['errors?'] = x['errors?'].replace('OK',1).replace('ERROR',0)
        x['errors?'] = x['errors?'].astype(int) 
        #x['has_chip'] = x['has_chip'].replace('YES',1).replace('NO',0)
        ### remove $
        
        for mc in self.money_col:
            x[mc] = x[mc].replace('[\$,]', '', regex=True).astype(float)
                ## split the date
        for col in self.date_col:
            temp = all_data[col].str.split("/",n=1,expand = True)
            x[col+'_month'] = temp[0]
            x[col+'_year'] = temp[1]
            x = x.drop(columns= col)
            x[col+'_month'] = x[col+'_month'].astype(int)
            x[col+'_year'] = x[col+'_year'].astype(int)
        for col in x.select_dtypes(object).columns:
            x[col] = x[col].astype('category')
        
        
        
        return x
    def GLMM_fit(self , x , encode_feature , y):
        
        self.GLMM[encode_feature] = ce.glmm.GLMMEncoder()
        self.GLMM[encode_feature].fit(x[encode_feature] , y )
        
        
    def GLMM_encode(self , x , encode_feature):
        return  self.GLMM[encode_feature].transform(x[encode_feature])
    
    def fit_transform(self, data):
        
        x = self.common_transform(data)
        x['mcc_fre'] = 0
        self.user_mcc_freq = {}
        for user in x["user_id"].unique():
            user_fre = {}
            for mcc in x['mcc'].unique():
                user_fre[mcc] = (x[x['user_id'] == user][x['mcc']==mcc] ).shape[0]  /x[ x["user_id"] ==user].shape[0] 
            self.user_mcc_freq[user] = user_fre
        self.user_mcc_freq = pd.DataFrame.from_dict( self.user_mcc_freq)
        
        
        
        for user in x["user_id"].unique():
            user_fre = {}
            for mcc in x['mcc'].unique():
                x['mcc_fre'] = np.where( np.logical_and(x['user_id'] == user , x['mcc']==mcc) , self.user_mcc_freq.loc[mcc , user]  ,x['mcc_fre'])
                #x.loc[x['user_id'] == user][x['mcc']==mcc]['mcc_fre'] = self.user_mcc_freq.loc[mcc , user] 
        
        
        self.GLMM_fit(x , "user_id" , x['amount'])
        x["glmm_amount_user_id"] = self.GLMM_encode(x , "user_id")
        
        cate_feature = x.dtypes
        self.cate_feature = list(cate_feature[cate_feature == "category"].index)
        self.label_emb = {}
        
        
        
        
        for feature in self.cate_feature:
            self.label_emb[feature] = x[[feature, 'is_fraud?']].groupby(feature).mean()['is_fraud?']
        
        self.price_avg = x[['amount','user_id']].groupby('user_id').mean()['amount']
        
        x['price_avg'] = x['user_id'].copy()
        x['price_avg'] = self.price_avg.loc[x['price_avg']].values 
        
        for feature in self.cate_feature:
            x[feature] = self.label_emb[feature].loc[x[feature]].values 
        for feat__ in self.user_emb_feature:
            self.GLMM_fit(x , "user_id" ,x[feat__] )
            x[feat__] = self.GLMM_encode(x , "user_id")
        ## average spend
        
        
        return x
    def expand(self, feature_mapping, test_data):
        return pd.DataFrame([test_data.value_counts(), feature_mapping], index = ["test", "train"]).T["train"]
    def transform(self, data):
        x = self.common_transform(data)
        x['mcc_fre'] = 0
        for user in x["user_id"].unique():
            user_fre = {}
            for mcc in x['mcc'].unique():
                x['mcc_fre'] = np.where( np.logical_and(x['user_id'] == user , x['mcc']==mcc) , self.user_mcc_freq.loc[mcc , user]  ,x['mcc_fre'])
                
        x["glmm_amount_user_id"] = self.GLMM_encode(x , "user_id")
        ## ---- price_avg
        x['price_avg'] = x['user_id'].copy()
        x['price_avg'] = self.price_avg.loc[x['price_avg']].values 
        
        for feature in self.cate_feature:
            self.label_emb[feature] = self.expand(self.label_emb[feature], x[feature])
            x[feature] = self.label_emb[feature].loc[x[feature]].values
        for feat__ in self.user_emb_feature:
            x[feat__] = self.GLMM_encode(x , "user_id")
        return x

In [4]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

# Preprocess

In [5]:
pre = preprocessor()
train_pre = pre.fit_transform(train)
x , y = train_pre.drop(columns = ['is_fraud?']) ,train_pre['is_fraud?']


test_pre = pre.transform(test)


C:\Users\zxc59\AppData\Local\Temp\ipykernel_21188\1411446235.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.

  user_fre[mcc] = (x[x['user_id'] == user][x['mcc']==mcc] ).shape[0]  /x[ x["user_id"] ==user].shape[0]


# Setting up grid search parameters

In [6]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
    
        'max_depth': [4, 5, 6 , 7 ,8, 10],
        "scale_pos_weight" : [2,5,8,10] , 
        'n_estimators' : [512,1024]
        }

In [7]:
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 1001)

xgb = XGBClassifier( objective='binary:logistic')
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='f1', n_jobs=4, cv=skf.split(x,y), verbose=3 )

start_time = timer(None) # timing starts from this point for "start_time" variable
grid.fit(x, y)
timer(start_time)

Fitting 5 folds for each of 3 candidates, totalling 15 fits



 Time taken: 0 hours 1 minutes and 1.3 seconds.


In [8]:
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results-01.csv', index=False)

# y_test = grid.best_estimator_.predict_proba(test)
# results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
# results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)



 All results:

{'mean_fit_time': array([13.54119697, 13.45896134, 12.29039888]), 'std_fit_time': array([0.07398839, 0.05052266, 0.94206049]), 'mean_score_time': array([0.0666008 , 0.06427259, 0.06504335]), 'std_score_time': array([0.00537936, 0.00238943, 0.00698836]), 'param_min_child_weight': masked_array(data=[1, 5, 10],

             mask=[False, False, False],

       fill_value='?',

            dtype=object), 'params': [{'min_child_weight': 1}, {'min_child_weight': 5}, {'min_child_weight': 10}], 'split0_test_score': array([0.58704137, 0.58456134, 0.58710937]), 'split1_test_score': array([0.59264318, 0.59273012, 0.59195627]), 'split2_test_score': array([0.59217661, 0.59614079, 0.59103669]), 'split3_test_score': array([0.59553925, 0.60373364, 0.59946029]), 'split4_test_score': array([0.5998642 , 0.59430674, 0.59767667]), 'mean_test_score': array([0.59345292, 0.59429452, 0.59344786]), 'std_test_score': array([0.00421544, 0.00615935, 0.00452211]), 'rank_test_score': array([2, 1, 3]

# Make the submission

In [15]:
test_pre = pre.transform(test)
y_test = grid.best_estimator_.predict(test_pre)
test['result'] = y_test
test['result'].to_csv("best_from_grid.csv",header=False)